In [1]:
!pip install -q pyjwt

In [2]:
import requests, jwt
from IPython.display import JSON

CATALOG_URL = "http://lakekeeper:8181/catalog"
MANAGEMENT_URL = "http://lakekeeper:8181/management"
KEYCLOAK_TOKEN_URL = "http://keycloak:8080/realms/myrealm/protocol/openid-connect/token"

# Sign in

In [ ]:
# Login to Keycloak
CLIENT_ID = "trino"
CLIENT_SECRET = "AK48QgaKsqdEpP9PomRJw7l2T7qWGHdZ"

response = requests.post(
    url=KEYCLOAK_TOKEN_URL,
    data={
        "grant_type": "client_credentials",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "scope": "lakekeeper"
    },
    headers={"Content-type": "application/x-www-form-urlencoded"},
)
response.raise_for_status()
access_token = response.json()['access_token']

# Lets inspect the token we got to see that our application name is available:
JSON(jwt.decode(access_token, options={"verify_signature": False}))

# Creating a Warehouse

In [ ]:
response = requests.post(
    url=f"{MANAGEMENT_URL}/v1/warehouse",
    headers={
        "Authorization": f"Bearer {access_token}"
    },
    json={
      "warehouse-name": "demo",
      "storage-profile": {
        "type": "s3",
        "bucket": "examples",
        "key-prefix": "initial-warehouse",
        "endpoint": "http://minio:9000",
        "region": "local-01",
        "path-style-access": True,
        "flavor": "minio",
        "sts-enabled": True
      },
      "storage-credential": {
        "type": "s3",
        "credential-type": "access-key",
        "aws-access-key-id": "minioadmin",
        "aws-secret-access-key": "minioadmin"
      }
    }
)
response.raise_for_status()
JSON(response.json())    
    